Importamos las librerías necesarias.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Cargamos las tablas de la base de datos en un DataFrame de pandas.

In [2]:
# Conectarse a la base de datos SQLite
conexion = sqlite3.connect('bookmaker.db')

# Leer las tablas en DataFrames de pandas
df_apuestas = pd.read_sql_query("SELECT * FROM apuestas", conexion)
df_partidos = pd.read_sql_query("SELECT * FROM partidos", conexion)
df_equipos = pd.read_sql_query("SELECT * FROM equipos", conexion)
df_empresa = pd.read_sql_query("SELECT * FROM empresa", conexion)
df_clientes = pd.read_sql_query("SELECT * FROM clientes", conexion)
df_cuotas = pd.read_sql_query("SELECT * FROM cuotas", conexion)

# Cerrar la conexión
conexion.close()

Vemos las 5 primeras filas de cada tabla para entender su estructura.

In [3]:
df_apuestas.head()

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia
0,1,2021-12-15 01:08:30.211674,300.0,2.0,1,88,891.0
1,2,2021-12-15 01:08:30.211674,300.0,29.0,1,88,891.0
2,3,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0
3,4,2021-12-15 01:35:34.965516,300.0,29.0,1,88,891.0
4,5,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0


In [4]:
df_partidos.head()

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15 00:51:23.181740,29,2,0,None
1,2,2021-12-15 00:51:23.181740,31,19,0,None
2,3,2021-12-15 00:51:23.181740,25,24,0,None
3,4,2021-12-15 00:51:23.181740,11,6,0,None
4,5,2021-12-15 00:51:23.181740,4,5,0,None


In [5]:
df_equipos.head()

,id,nombre,escudo,pais,puntaje,activado
0,1,Ajax,ajax.png,Paises Bajos,14.0,1
1,2,Atalanta,atalanta.png,Italia,6.0,1
2,3,Atlético,atletico.png,España,11.0,1
3,4,Barcelona,barcelona.png,España,17.0,1
4,5,Bayern,bayern.png,Alemania,16.0,1


In [6]:
df_empresa.head()

,id,razon_social,email,activado


In [7]:
df_clientes.head()

,id,nombre,apellido,email,activado
0,1,Lope,Perales,pascualperera@vila.es,1
1,2,Isaura,Canet,gmerino@rocamora.es,1
2,3,Olga,Chaves,amayaprats@llado.com,1
3,4,Celestino,Hoyos,chedelgado@moraleda-pintor.com,1
4,5,Miguel,Milla,inigoprat@gmail.com,1


In [8]:
df_cuotas.head()

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5


A continuación vamos a utilizar la tabla cuotas para predecir el resultado de la tabla partidos. De forma que daremos el resultado cuya cuota sea la más baja en cada partido.

Antes que nada, vamos a limpiar los datos. En primer lugar, vamos a eliminar las filas que tengan valores nulos en las columnas en las que no deberian tenerlos. 

In [9]:
# Eliminamos la columna finalizado del DataFrame de partidos ya que no nos aporta información relevante
df_partidos.drop(['finalizado'], axis=1, inplace=True)

# Eliminamos las filas con valores nulos del DataFrame de cuotas
df_cuotas.dropna(inplace=True)

# Eliminamos las filas con valores nulos del DataFrame de partidos. La columna ganador tiene valores nulos siempre asi que en esa columna no comprobamos si hay valores nulos
df_partidos.dropna(inplace=True, subset=['id','fecha','equipo_local','equipo_visistante'])

Otro problema que puede haber en estos dos dataframes es que haya partidos duplicados.

In [10]:
# Nos aseguramos que no haya partidos duplicados en el DataFrame de cuotas
df_cuotas.drop_duplicates(subset='partido_id', keep='first', inplace=True)

# Nos aseguramos que no haya partidos duplicados en el DataFrame de partidos
df_partidos.drop_duplicates(subset='id', keep='first', inplace=True)

In [11]:
# Encontrar el tipo de resultado con la cuota más baja (1, X, o 2)
df_cuotas['resultado'] = df_cuotas[['local', 'empate', 'visitante']].idxmin(axis=1)

# Crear un nuevo DataFrame con las columnas necesarias
df_relacion = df_cuotas[['partido_id', 'resultado']]

# Fusionar los DataFrames en función de la columna 'partido_id'
df_partidos = pd.merge(df_partidos, df_relacion, left_on='id', right_on='partido_id', how='left')

# Mapear el resultado a la columna 'ganador' en el DataFrame de partidos
mapeo_resultados = {'local': '1', 'empate': 'X', 'visitante': '2'}
df_partidos['ganador'] = df_partidos['resultado'].map(mapeo_resultados)

# Eliminar columnas adicionales
df_partidos.drop(['partido_id', 'resultado'], axis=1, inplace=True)
df_cuotas.drop(['resultado'], axis=1, inplace=True)

Ahora vamos a actualizar la columna puntaje del csv equipos con los puntos que tendría cada equipo de acuerdo con nuestras cuotas.

En primer lugar vamos a limpiar ese dataframe.

In [12]:
# Eliminamos las columna activado y escudo del DataFrame equipos ya que no nos aporta información relevante
df_equipos.drop(['activado'], axis=1, inplace=True)
df_equipos.drop(['escudo'], axis=1, inplace=True)

# Eliminamos las filas con valores nulos en la columna nombre del DataFrame equipos
df_equipos.dropna(inplace=True, subset=['nombre'])

# La columna puntaje ahora mismo no está actualizada con los valores que queremos darle, por lo que de momento la igualemos en todas las filas a 0
df_equipos['puntaje'] = 0

Ahora asignamos los puntajes. Para ello, vamos a recorrer el dataframe de partidos y vamos a asignar 3 puntos al equipo que gane, 1 al que empate y 0 al que pierda. A continuacion, vamos a actualizar el dataframe de equipos sumando los puntajes que acabamos de calcular para cada equipo.

In [13]:
# Crear una nueva columna 'puntos_local' y 'puntos_visitante' en el DataFrame de partidos
df_partidos['puntos_local'] = df_partidos['ganador'].apply(lambda x: 3 if x == '1' else 1 if x == 'X' else 0)
df_partidos['puntos_visitante'] = df_partidos['ganador'].apply(lambda x: 3 if x == '2' else 1 if x == 'X' else 0)

# Sumar los puntos por equipo en dos nuevos dataframes, uno para los locales y otro para los visitantes
df_puntos_por_equipo_local = df_partidos.groupby('equipo_local')['puntos_local'].sum().reset_index()
df_puntos_por_equipo_visistante = df_partidos.groupby('equipo_visistante')['puntos_visitante'].sum().reset_index()

# Fusionar los puntos por equipo en un nuevo DataFrame único
df_puntos_totales_por_equipo = df_puntos_por_equipo_local.merge(df_puntos_por_equipo_visistante, left_on='equipo_local', right_on='equipo_visistante', how='outer')

# Calcular el puntaje total de cada equipo sumando las columnas 'puntos_local' y 'puntos_visitante' del nuevo dataframe puntos_totales_por_equipo
df_puntos_totales_por_equipo['puntaje_total'] = df_puntos_totales_por_equipo['puntos_local'].fillna(0) + df_puntos_totales_por_equipo['puntos_visitante'].fillna(0)

# Actualizar la columna 'puntaje' en el DataFrame de equipos
df_equipos['puntaje'] = df_equipos['id'].map(df_puntos_totales_por_equipo.set_index('equipo_local')['puntaje_total'])

In [14]:
# Aqui podemos ver las 5 primeras dilas del nuevo dataframe creado con los puntos de cada equipo
df_puntos_totales_por_equipo.head()

,equipo_local,puntos_local,equipo_visistante,puntos_visitante,puntaje_total
0,1,3,1,3,6
1,2,0,2,0,0
2,3,0,3,0,0
3,4,6,4,6,12
4,5,3,5,3,6


In [23]:
# Y aqui podemos ver las 32 primeras filas del dataframe de equipos con la columna puntaje actualizada
df_equipos.head(32)

,id,nombre,pais,puntaje
0,1,Ajax,Paises Bajos,6
1,2,Atalanta,Italia,0
2,3,Atlético,España,0
3,4,Barcelona,España,12
4,5,Bayern,Alemania,6
5,6,Benfica,Portugal,0
6,7,Besiktas,Turquía,0
7,8,Chelsea,Inglaterra,0
8,9,Club Brugge,Bélgica,18
9,10,Dortmund,Alemania,18
